In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

from statsmodels.tsa.stattools import grangercausalitytests

# Global variables and paths

In [2]:
company = 'sunpower'
ticker = 'SPWR'

In [3]:
path_stocks = '../../download_stock_data/' + ticker + '.csv'
path_twitter_sentiment = '../../../../data/team/NLTK Time Series/sentiment_nltk_fixed_final.csv'

# Import Data

## Import Stock Data

In [4]:
stock = pd.read_csv(path_stocks)
stock['Date'] = pd.to_datetime(stock['timestamp'])
stock.set_index('Date', inplace=True);
stock.drop(['timestamp'],axis=1, inplace=True)
stock_close_daily = pd.DataFrame(stock['close'])

## Import Twitter Data

In [5]:
twitter_sentiment = pd.read_csv(path_twitter_sentiment)
twitter_sentiment.drop('Unnamed: 0', axis=1, inplace=True)
twitter_sentiment['Date'] = pd.to_datetime(twitter_sentiment['CreatedAt'].str[:10], errors='coerce')
twitter_sentiment.set_index('Date',inplace=True);
twitter_sentiment.drop('CreatedAt', axis=1, inplace=True)

# Select Company

In [6]:
# select twitter_sentiment data based on company
twitter_sentiment_company = twitter_sentiment[twitter_sentiment['company'] == company]

# Create daily tweet count 

In [7]:
tweets_daily = twitter_sentiment_company.groupby(['Date']).count()
tweets_daily = pd.DataFrame(tweets_daily['compound'])
tweets_daily.rename(columns={'compound':'Daily Tweets'},inplace=True)

# Create Sentiment data

In [8]:
sentiment = twitter_sentiment_company[['neg','pos','compound']]

In [9]:
sentiment_daily = pd.DataFrame(sentiment['neg']).groupby(['Date']).mean()

In [10]:
sentiment_daily.sort_index(ascending=False);

In [11]:
sentiment_daily['pos'] = pd.DataFrame(sentiment['pos']).groupby(['Date']).mean()

In [12]:
sentiment_daily['compound'] = pd.DataFrame(sentiment['compound']).groupby(['Date']).mean()

In [13]:
sentiment_daily.sort_index(ascending=False)

,neg,pos,compound
Date,,,
2021-04-01,0.000000,0.346000,0.571900
2021-03-31,0.020127,0.130732,0.239786
2021-03-30,0.014551,0.169077,0.327376
2021-03-29,0.029778,0.113750,0.229036
2021-03-28,0.010474,0.176632,0.375642
...,...,...,...
2011-01-06,0.000000,0.093778,0.157900
2011-01-05,0.000000,0.000000,0.000000
2011-01-04,0.000000,0.119733,0.209133


In [14]:
# sentiment_daily = sentiment.groupby(['Date']).count()

# Merge Data

In [15]:
data = pd.merge(tweets_daily, sentiment_daily, how='left', left_on=['Date'], right_on=['Date'] )
data = pd.merge(data, stock_close_daily, how='left', left_on=['Date'], right_on=['Date'] )
data.dropna(inplace=True)
data.rename(columns={'Daily Tweets':'daily_tweets','close':'closing_price', 'compound':'compound_sentiment', 'neg':'neg_sentiment','pos':'pos_sentiment'}, inplace=True)

# Export Data 

In [16]:
# name of output csv
output = "./data_" + company + "_"  + ticker + ".csv"

In [17]:
data.to_csv(output)

In [18]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,3,0.000000,0.253000,0.385867,13.0600
2011-01-04,15,0.000000,0.119733,0.209133,13.6300
2011-01-05,3,0.000000,0.000000,0.000000,13.8600
2011-01-06,18,0.000000,0.093778,0.157900,13.6275
2011-01-07,5,0.022000,0.250600,0.369080,13.4400
...,...,...,...,...,...
2021-03-26,52,0.022558,0.140096,0.201854,32.8700
2021-03-29,36,0.029778,0.113750,0.229036,29.2400
2021-03-30,78,0.014551,0.169077,0.327376,31.5200


# Normalize Data

In [19]:
def normalize_daily_tweets(df):
    max_count = df['daily_tweets'].max()
    min_count = df['daily_tweets'].min()
    df['daily_tweets'] = (df['daily_tweets'] - min_count) / (max_count - min_count)
    return df


def normalize_neg_sentiment(df):
    max_count = df['neg_sentiment'].max()
    min_count = df['neg_sentiment'].min()
    df['neg_sentiment'] = (df['neg_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_pos_sentiment(df):
    max_count = df['pos_sentiment'].max()
    min_count = df['pos_sentiment'].min()
    df['pos_sentiment'] = (df['pos_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_compound_sentiment(df):
    max_count = df['compound_sentiment'].max()
    min_count = df['compound_sentiment'].min()
    df['compound_sentiment'] = (df['compound_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_closing_price(df):
    max_count = df['closing_price'].max()
    min_count = df['closing_price'].min()
    df['closing_price'] = (df['closing_price'] - min_count) / (max_count - min_count)
    return df

In [20]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,3,0.000000,0.253000,0.385867,13.0600
2011-01-04,15,0.000000,0.119733,0.209133,13.6300
2011-01-05,3,0.000000,0.000000,0.000000,13.8600
2011-01-06,18,0.000000,0.093778,0.157900,13.6275
2011-01-07,5,0.022000,0.250600,0.369080,13.4400
...,...,...,...,...,...
2021-03-26,52,0.022558,0.140096,0.201854,32.8700
2021-03-29,36,0.029778,0.113750,0.229036,29.2400
2021-03-30,78,0.014551,0.169077,0.327376,31.5200


In [21]:
data_normalized = normalize_daily_tweets(data)

In [22]:
data_normalized = normalize_neg_sentiment(data)

In [23]:
data_normalized = normalize_pos_sentiment(data)

In [24]:
data_normalized = normalize_compound_sentiment(data)

In [25]:
data_normalized = normalize_closing_price(data)

In [26]:
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.002328,0.000000,0.408813,0.610413,0.184912
2011-01-04,0.016298,0.000000,0.193472,0.442613,0.196258
2011-01-05,0.002328,0.000000,0.000000,0.244050,0.200836
2011-01-06,0.019790,0.000000,0.151532,0.393969,0.196208
2011-01-07,0.004657,0.090246,0.404935,0.594475,0.192476
...,...,...,...,...,...
2021-03-26,0.059371,0.092534,0.226376,0.435701,0.579220
2021-03-29,0.040745,0.122151,0.183804,0.461509,0.506967
2021-03-30,0.089639,0.059691,0.273205,0.554878,0.552349


In [27]:
output_normalized = "./data_" + company + "_"  + ticker + "_normalized.csv"
data.to_csv(output_normalized)
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.002328,0.000000,0.408813,0.610413,0.184912
2011-01-04,0.016298,0.000000,0.193472,0.442613,0.196258
2011-01-05,0.002328,0.000000,0.000000,0.244050,0.200836
2011-01-06,0.019790,0.000000,0.151532,0.393969,0.196208
2011-01-07,0.004657,0.090246,0.404935,0.594475,0.192476
...,...,...,...,...,...
2021-03-26,0.059371,0.092534,0.226376,0.435701,0.579220
2021-03-29,0.040745,0.122151,0.183804,0.461509,0.506967
2021-03-30,0.089639,0.059691,0.273205,0.554878,0.552349


In [28]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.002328,0.000000,0.408813,0.610413,0.184912
2011-01-04,0.016298,0.000000,0.193472,0.442613,0.196258
2011-01-05,0.002328,0.000000,0.000000,0.244050,0.200836
2011-01-06,0.019790,0.000000,0.151532,0.393969,0.196208
2011-01-07,0.004657,0.090246,0.404935,0.594475,0.192476
...,...,...,...,...,...
2021-03-26,0.059371,0.092534,0.226376,0.435701,0.579220
2021-03-29,0.040745,0.122151,0.183804,0.461509,0.506967
2021-03-30,0.089639,0.059691,0.273205,0.554878,0.552349
